```
《动手学习深度学习》：
    最新书籍：http://zh.d2l.ai/chapter_preface/index.html
    B站：https://space.bilibili.com/1567748478/channel/seriesdetail?sid=358497
```

# 深度神经网络

## 神经网络搭建八股-模块化设计

In [7]:
from tensorflow.examples.tutorials.mnist import input_data

minist = input_data.read_data_sets('./minist_data', one_hot=True)
minist

Extracting ./minist_data\train-images-idx3-ubyte.gz
Extracting ./minist_data\train-labels-idx1-ubyte.gz
Extracting ./minist_data\t10k-images-idx3-ubyte.gz
Extracting ./minist_data\t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001D9F526F828>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001D9F526F860>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001D9F526F978>)

In [54]:
def get_weights(shape, REGULARIZER):
    w = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    if REGULARIZER != None:
        tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(REGULARIZER)(w)) # 添加正则化
    return w

def get_bias(shape):
    '''bias 不进行正则化'''
    b = tf.Variable(tf.zeros(shape=shape))
    return b

def forward(x, REGULARIZER): # 数据的前向传播
    '''
    搭建网络，设计网络结构
    按照计算步骤，从神经网络的各层依次进行计算（传播）
    '''
    # 隐藏层
    n_hidden = 100
    w_h = get_weights(shape=[784, n_hidden], REGULARIZER=REGULARIZER)
    b_h = get_bias(n_hidden)
    a_h = tf.nn.relu(tf.matmul(x, w_h) + b_h)
    # 输出层
    n_out = 10
    w_o = get_weights(shape=[n_hidden, n_out], REGULARIZER=REGULARIZER)
    b_o = get_bias(n_out)
    z_o = tf.matmul(a_h, w_o) + b_o # 输出层不过激活函数
    return z_o

def backpropagate(): # 误差的反向传播
    '''
    训练网络，优化网络参数
    反向求导，进行误差的反向传播
    '''
    # REGULARIZER = None # 先不正则化看看
    REGULARIZER = 0.01
    BATCH_SIZE= 200 # 每次提交的批量样本量
    EPOCHS = 500 # 迭代数

    ############
    # 前向计算
    x = tf.placeholder(tf.float32, shape=(None, 784)) # 自变量x
    y = tf.placeholder(tf.float32, shape=(None, 10)) # 真实y值
    y_hat = forward(x, REGULARIZER) # 计算的y值

    ############
    # loss function
    mse = tf.reduce_mean(tf.square(y_hat - y)) # 定义损失函数: 1. 均方误差-MSE 2. 自定义 3. 交叉熵
    mse_regul = mse + tf.add_n(tf.get_collection('losses')) # 正则化

    ############
    # opt优化
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss=mse_regul)

    ############
    # 开始会话
    with tf.Session() as sess:
        # 初始化所有变量
        init = tf.global_variables_initializer()
        sess.run(init)
        
        cost_list = []  
        for i in range(EPOCHS):
            xs, ys = minist.train.next_batch(BATCH_SIZE)
            cost, _ = sess.run([mse, train_step], {x:xs, y:ys})
            cost_list.append(cost)
            # if not i % 1000:
            #     print(f"{i}) cost: {cost}")
    return cost_list

if __name__ == "__main__":
    # backpropagate()
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(backpropagate())
    plt.xlabel('epoch')
    plt.ylabel('mse')

ValueError: inputs must be a list of at least one Tensor with the same dtype and shape

In [53]:
# REGULARIZER = None # 先不正则化看看
REGULARIZER = 0.01
BATCH_SIZE= 200 # 每次提交的批量样本量
EPOCHS = 500 # 迭代数

def backpropagate(): # 误差的反向传播
    '''
    训练网络，优化网络参数
    反向求导，进行误差的反向传播
    '''
    x = tf.placeholder(tf.float32, shape=(None, 784)) # 自变量x
    y = tf.placeholder(tf.float32, shape=(None, 10)) # 真实y值

    y_hat = forward(x, REGULARIZER) # 计算的y值

    # loss:
    #       1. 均方误差：MSE
    mse = tf.reduce_mean(tf.square(y_hat - y)) # 定义损失函数mse
    mse_regul = mse + tf.add_n(tf.get_collection('losess'))
    #       2. 自定义
    #       3. 交叉熵
    # ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y, 1))
    # cem = tf.reduce_mean(ce)

    # loss正则化：
    # mse_regul = mse + tf.add_n(tf.get_collection('losses')) # 正则化loss_mse
    # loss_mse_regul = cem + tf.add_n(tf.get_collection('losses')) # 正则化loss_mse

    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss=mse_regul)

    with tf.Session() as sess:
        # 初始化所有变量
        init = tf.global_variables_initializer()
        sess.run(init)
        
        cost_list = []  
        for i in range(EPOCHS):
            xs, ys = minist.train.next_batch(BATCH_SIZE)
            cost, _ = sess.run([mse, train_step], {x:xs, y:ys})
            cost_list.append(cost)
            # if not i % 1000:
            #     print(f"{i}) cost: {cost}")
    return cost_list

if __name__ == "__main__":
    # backpropagate()
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(backpropagate())
    plt.xlabel('epoch')
    plt.ylabel('mse')

ValueError: inputs must be a list of at least one Tensor with the same dtype and shape

In [ ]:
def predict():
    return

predict(minist.test)

# 卷积神经网络-CNN

## LeNet5

# 循环神经网络-RNN

## Transform